<a href="https://colab.research.google.com/github/alunfes/GoogleColabProjects/blob/main/GACartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random


In [2]:
class Agent(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Agent, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        return self.fc(x)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
def mutate(agent, mutation_rate=0.1):
    with torch.no_grad():
        for param in agent.parameters():
            if len(param.shape) == 2:  # weights of linear layer
                for i in range(param.shape[0]):
                    for j in range(param.shape[1]):
                        if random.random() < mutation_rate:
                            param[i][j] += np.random.normal(0, 0.1)
            elif len(param.shape) == 1:  # biases of linear layer
                for i in range(param.shape[0]):
                    if random.random() < mutation_rate:
                        param[i] += np.random.normal(0, 0.1)

def crossover(agent1, agent2):
    child_agent = Agent(input_size, hidden_size, output_size)
    with torch.no_grad():
        for child_param, param1, param2 in zip(child_agent.parameters(), agent1.parameters(), agent2.parameters()):
            child_param.data.copy_(param1.data if random.random() > 0.5 else param2.data)
    return child_agent

def evaluate_agent(agent, env, episodes=5):
    total_reward = 0.0
    for _ in range(episodes):
        state = env.reset()
        done = False
        while not done:
            state = torch.from_numpy(state).float().unsqueeze(0)
            action_probabilities = agent(state)
            action = torch.argmax(action_probabilities).item()
            state, reward, done, _ = env.step(action)
            total_reward += reward
    return total_reward / episodes


In [4]:
env = gym.make('CartPole-v1')
input_size = env.observation_space.shape[0]
hidden_size = 64
output_size = env.action_space.n

# 初期個体群の生成
population_size = 50
agents = [Agent(input_size, hidden_size, output_size) for _ in range(population_size)]

# GAの繰り返し
generations = 10
for generation in range(generations):
    # 評価
    scores = [evaluate_agent(agent, env) for agent in agents]

    # 選択
    sorted_indices = np.argsort(scores)[::-1]
    agents = [agents[i] for i in sorted_indices[:10]]  # トップ10を選択

    # 交叉と突然変異
    while len(agents) < population_size:
        agent1, agent2 = random.sample(agents[:10], 2)
        child = crossover(agent1, agent2)
        mutate(child)
        agents.append(child)

    # 世代の進行を報告
    print(f"Generation {generation}, Top Score: {scores[sorted_indices[0]]}")

# 最適なエージェントの選択
best_agent = agents[np.argmax([evaluate_agent(agent, env) for agent in agents])]


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Generation 0, Top Score: 60.6
Generation 1, Top Score: 77.2
Generation 2, Top Score: 180.2
Generation 3, Top Score: 108.6
Generation 4, Top Score: 243.4
Generation 5, Top Score: 490.8
Generation 6, Top Score: 486.6
Generation 7, Top Score: 500.0
Generation 8, Top Score: 500.0
Generation 9, Top Score: 500.0
Generation 10, Top Score: 500.0
Generation 11, Top Score: 500.0
Generation 12, Top Score: 500.0
Generation 13, Top Score: 500.0
Generation 14, Top Score: 500.0
Generation 15, Top Score: 500.0
Generation 16, Top Score: 500.0
Generation 17, Top Score: 500.0
Generation 18, Top Score: 500.0
Generation 19, Top Score: 500.0


KeyboardInterrupt: ignored